<a href="https://colab.research.google.com/github/sorielkobayashi/Custom-Spleeter-Colab/blob/main/Neural%20Audio%20Stem%20Separation%20(FLAC%2024bit/48kHz%20%26%207zip).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Spleeter Colab

Esta es una extensión para [Deezer Spleeter](https://github.com/deezer/spleeter) que se ejecuta en Google Colab, utilizando Google Drive como almacenamiento de datos.

El valor de `input` puede ser
1.   Ruta a un archivo de audio individual ubicado en su Google Drive.
2.   Ruta a un directorio en su Google Drive que contenga varios archivos de audio. Cada archivo de audio encontrado en el directorio será procesado individualmente.
3.   URL individual de YouTube
4.   Múltiples URLs de YouTube separadas por espacios. Cada vídeo de YouTube será procesado individualmente.

La entrada será separada en 4 stems: voz, batería, bajo y otros. Los stems se guardarán como archivos WAV en su Google Drive. Introduzca `output_dir` relativo a su raíz de Google Drive.

In [8]:
#@title Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.
# <br><center><img src='https://raw.githubusercontent.com/dropcreations/7zip-in-Google-Colab/main/7zip-Logo.png' height="50" alt="7zip-logo"/></center>
# <center><h3><b>Install 7-Zip</b></h3></center><br>
from IPython.display import clear_output
!sudo apt update
!sudo apt install p7zip-full p7zip-rar unrar rar
clear_output()
print("Successfully Installed.")

import os
from google.colab import drive, output
drive.mount('/content/drive')
drive_root = '/content/drive/MyDrive/'

# inhagcutils
!pip -q install import-ipynb
!curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *
!gsutil -q -m cp -R gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json /content/cfg.json

#@title Install
!apt -q install ffmpeg
!pip -q install spleeter

output.clear()
print('Using config gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json')
op(c.ok, 'Setup finished.')


Successfully Installed.


KeyboardInterrupt: ignored

In [9]:
#@title Separar

input = "/content/Mundo De Caramelo - Danna Paola.flac" #@param {type:"string"}
output_dir = "/s7ems" #@param {type:"string"}

#----------------------------------------------------------------------------#

import os
from os.path import isfile, join
from glob import glob

input_type = check_input_type(input)
tmp_dir = '/content/tmp/'
seps_dir = fix_path(drive_root+output_dir)
#seps_dir = '/content/tmp/seps'

if not os.path.isdir(tmp_dir):
  create_dirs([tmp_dir])
# if not os.path.isdir(output_dir):
#   create_dirs([seps_dir])

if output_dir == '' or output_dir == 'same':
  output_dir = fix_path(drive_root+path_dir(input))
else:
  output_dir = fix_path(drive_root+output_dir)
  if not os.path.isdir(output_dir):
    create_dirs([output_dir])

if input_type == 'unknown':
  input = drive_root+input
  input_type = check_input_type(input)
  if input_type=='dir':
    input = fix_path(input)

if input_type == 'youtube':
  !pip -q install youtube-dl
  !youtube-dl --restrict-filenames -x --no-continue --audio-format flac -o "{tmp_dir}%(title)s.%(ext)s" {input}

from spleeter.separator import Separator
separator = Separator('/content/cfg.json')

if input_type == "dir" or input_type == "youtube":
  if input_type == "dir":
    target = input
  if input_type == "youtube":
    target = dir_tmp
  file_list = list_audio(target)
  for audiofile in file_list:
    op(c.title, '\nSeparate '+audiofile+' to '+seps_dir.replace(drive_root, '')+basename(input))
    # Spleeter Python API is outdated for Colab
    # separator.separate_to_file(audiofile, seps_dir)
    !spleeter separate -p /content/cfg.json -c flac -d 666 -o "{output_dir}" "{audiofile}"
    op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(audiofile)+'/')
elif input_type == "file":
  # Spleeter Python API is outdated for Colab
  # separator.separate_to_file(input, seps_dir)
  !spleeter separate -p /content/cfg.json -c flac -d 666 -o "{seps_dir}" "{input}"
  op(c.ok, 'FIN.', 'Files saved under directory '+output_dir.replace(drive_root, '')+basename(input)+'/')
else:
  print("We're not sure what you want to separate. Please check the input field.")

tcmalloc: large alloc 1153531904 bytes == 0xa3746000 @  0x7ff12ad021e7 0x7ff0f71c116a 0x7ff0fa239be3 0x7ff0fa71c736 0x7ff0fa76c1b8 0x7ff0fa76ddb3 0x7ff0fa76f1e2 0x7ff0fa7710df 0x7ff0fa775cd2 0x7ff0fa776088 0x7ff0ee94864b 0x7ff0ee8ec060 0x7ff0ee8d6aae 0x7ff0fc634dd5 0x7ff0fc632b03 0x7ff0ef03633b 0x7ff12a6c66db 0x7ff12a9ff61f
tcmalloc: large alloc 2306973696 bytes == 0xcf746000 @  0x7ff12ad021e7 0x7ff0f71c116a 0x7ff0fa239be3 0x7ff0fa71c736 0x7ff0fa76c1b8 0x7ff0fa76ddb3 0x7ff0fa76f1e2 0x7ff0fa7710df 0x7ff0fa775cd2 0x7ff0fa776088 0x7ff0ee94864b 0x7ff0ee8ec060 0x7ff0ee8d6aae 0x7ff0fc634dd5 0x7ff0fc632b03 0x7ff0ef03633b 0x7ff12a6c66db 0x7ff12a9ff61f
tcmalloc: large alloc 2306973696 bytes == 0x1c1fa6000 @  0x7ff12ad021e7 0x7ff0f71c116a 0x7ff0fa239be3 0x7ff0fa71c736 0x7ff0fa76c1b8 0x7ff0fa76ddb3 0x7ff0fa76f1e2 0x7ff0fa7710df 0x7ff0fa775cd2 0x7ff0fa776088 0x7ff0ee94864b 0x7ff0ee8ec060 0x7ff0ee8d6aae 0x7ff0fc634dd5 0x7ff0fc632b03 0x7ff0ef03633b 0x7ff12a6c66db 0x7ff12a9ff61f
tcmalloc: large alloc

In [10]:
#@title Comprimir
import os

sourcePath = "/content/drive/MyDrive/s7ems/Mundo De Caramelo - Danna Paola" #@param {type:"string"}
compressType = "7z" #@param ["zip", "7z", "tar", "gzip", "bzip2", "xz", "wim"]
Password = "" #@param {type:"string"}
Split = "no" #@param ["no", "10m", "100m", "500m", "1g", "2g"] {allow-input: true}
compressLevel = 9 #@param {type:"slider", min:0, max:9, step:1}
saveToSourceLocation = True #@param {type:"boolean"}

commandLine = "-t" + compressType + " -mx=" + str(compressLevel)

if len(Password) > 0:
    commandLine = commandLine + " -p" + '"' + Password + '"'

if Split is not "no":
    commandLine = commandLine + " -v" + '"' + Split + '"'

if os.path.isfile(sourcePath) is True:
    sourceName = os.path.splitext(os.path.basename(os.path.abspath(sourcePath)))[0]
    sourceFolder = os.path.dirname(os.path.abspath(sourcePath))
else:
    sourceName = os.path.split(os.path.abspath(sourcePath))[1]
    sourceFolder = os.path.split(os.path.abspath(sourcePath))[0]

if saveToSourceLocation == True:
    compressPath = os.path.join(sourceFolder, "compressed")
    commandLine = commandLine + ' "' + compressPath + '"'
else:
    outputPath = input("outputPath: ")
    if outputPath.endswith('.zip') or outputPath.endswith('.7z') or outputPath.endswith('.tar') or outputPath.endswith('.gz') or outputPath.endswith('.bz2') or outputPath.endswith('.xz') or outputPath.endswith('.wim'):
        sourceName = os.path.splitext(os.path.basename(os.path.abspath(outputPath)))[0]
        sourceFolder = os.path.dirname(os.path.abspath(outputPath))
    else:
        if os.path.exists(outputPath) is False:
            os.makedirs(outputPath)
        sourceFolder = outputPath
    compressPath = os.path.join(sourceFolder, "compressed")
    commandLine = commandLine + ' "' + compressPath + '"'

if compressType == "gzip":
    formatFile = "gz"
elif compressType == "bzip2":
    formatFile = "bz2"
else:
    formatFile = compressType

!7z a {commandLine} "{sourcePath}"
saveFile = os.path.join(sourceFolder, sourceName)
compressPath = compressPath + '.' + formatFile
saveFile = saveFile + '.' + formatFile
os.rename(compressPath, saveFile)


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs AMD EPYC 7B12 (830F10),ASM,AES-NI)

Scanning the drive:
  0M Scan  /content/drive/MyDrive/s7ems/                                        1 folder, 4 files, 155291478 bytes (149 MiB)

Creating archive: /content/drive/MyDrive/s7ems/compressed.7z

Items to compress: 5

  0%      0% + Mundo De Caramelo - Danna Paola/bass.flac                                                  1% + Mundo De Caramelo - Danna Paola/bass.flac                                                  2% + Mundo De Caramelo - Danna Paola/bass.flac            